### Delta Table Creation

In [0]:
from delta.tables import *

DeltaTable.create(spark)\
    .tableName("employee_delta_creat")\
    .addColumn("Emp_id", "INT")\
    .addColumn("Emp_name", "STRING")\
    .addColumn("Gender", "STRING")\
    .addColumn("Salary", "INT")\
    .addColumn("Dept", "STRING")\
    .property("Description", "Delta Table Created for Demo")\
    .location("/FileStore/tables/delta2/createtables").execute()


Out[1]: <delta.tables.DeltaTable at 0x7f76e0c0fe20>

### SQL Style Insert

In [0]:
%sql
INSERT INTO employee_delta_creat VALUES(100, "Sabrina", "F", 10000, "Music")

num_affected_rows,num_inserted_rows
1,1


In [0]:
spark.sql("SElECT * FROM employee_delta_creat").display()

Emp_id,Emp_name,Gender,Salary,Dept
100,Sabrina,F,10000,Music


### DataFrame Insert

In [0]:
from pyspark.sql.types import StringType, IntegerType

employee_data = [(200, "Lara", "F", 8000, "Fashion")]
employee_schema = StructType([
  StructField("Emp_id", IntegerType(), False),
  StructField("Emp_name", StringType(), True),
  StructField("Gender", StringType(), True),
  StructField("Salary", IntegerType(), True),
  StructField("Dept", StringType(), True)])

df = spark.createDataFrame(employee_data, employee_schema)
df.display()

Emp_id,Emp_name,Gender,Salary,Dept
200,Lara,F,8000,Fashion


In [0]:
df.write.format("delta").mode("append").saveAsTable("employee_delta_creat")

In [0]:
%sql
SELECT * FROM employee_delta_creat

Emp_id,Emp_name,Gender,Salary,Dept
100,Sabrina,F,10000,Music
200,Lara,F,8000,Fashion


### DataFrame Insert Into Method

In [0]:
from pyspark.sql.types import StringType, IntegerType

employee_data = [(300, "Ariana", "F", 15000, "Composition")]
employee_schema = StructType([
  StructField("Emp_id", IntegerType(), False),
  StructField("Emp_name", StringType(), True),
  StructField("Gender", StringType(), True),
  StructField("Salary", IntegerType(), True),
  StructField("Dept", StringType(), True)])

df1 = spark.createDataFrame(employee_data, employee_schema)
df1.display()

Emp_id,Emp_name,Gender,Salary,Dept
300,Ariana,F,15000,Composition


In [0]:
df1.write.insertInto('employee_delta_creat', overwrite=False)

In [0]:
%sql
SELECT * FROM employee_delta_creat

Emp_id,Emp_name,Gender,Salary,Dept
300,Ariana,F,15000,Composition
100,Sabrina,F,10000,Music
200,Lara,F,8000,Fashion


### Insert using TempView

In [0]:
df1.createOrReplaceTempView("delta_data")

The below wont work because:
- %sql in Databricks only recognizes tables and views registered in the metastore, not temporary views created in a Python cell using createOrReplaceTempView

In [0]:
%sql
SELECT * FROM delta_data

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5507955271996596>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-5507955271996596>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBkZWx0YV9kYXRh").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databrick

In [0]:
df1.createOrReplaceGlobalTempView("delta_data")

In [0]:
%sql
SELECT * FROM global_temp.delta_data

Emp_id,Emp_name,Gender,Salary,Dept
300,Ariana,F,15000,Composition


Summary:
- Temporary views created in Python are not visible to %sql magic cells. Use spark.sql in Python or create a global temp view for %sql access.

In [0]:
%sql
INSERT INTO employee_delta_creat
SELECT * FROM global_temp.delta_data  -- Ariana will be Inserted again

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
SELECT * FROM employee_delta_creat

Emp_id,Emp_name,Gender,Salary,Dept
300,Ariana,F,15000,Composition
300,Ariana,F,15000,Composition
100,Sabrina,F,10000,Music
200,Lara,F,8000,Fashion
